In [4]:
import numpy as np
import time

# Objective function (Rastrigin)
def rastrigin(X):
    A = 10
    return A * len(X) + np.sum(X**2 - A * np.cos(2 * np.pi * X))

class Agent:
    def __init__(self, dim, bounds):
        self.position = np.random.uniform(bounds[0], bounds[1], dim)
        self.fitness = rastrigin(self.position)

    def update_position(self, new_position, bounds):
        self.position = np.clip(new_position, bounds[0], bounds[1])
        self.fitness = rastrigin(self.position)

class AdaptiveGWOSCSO:
    def __init__(self, obj_func, dim, bounds=(-5.12, 5.12), n_agents=30,
                 max_iter=100, stagnation_limit=20, recovery_duration=10):
        self.obj_func = obj_func
        self.dim = dim
        self.bounds = bounds
        self.n_agents = n_agents
        self.max_iter = max_iter
        self.stagnation_limit = stagnation_limit
        self.recovery_duration = recovery_duration

        self.population = np.random.uniform(bounds[0], bounds[1], (n_agents, dim))
        self.fitness = np.apply_along_axis(self.obj_func, 1, self.population)

        self.mode = "GWO"
        self.stagnation_counter = 0
        self.recovery_timer = 0

        self.best_idx = np.argmin(self.fitness)
        self.best_solution = self.population[self.best_idx].copy()
        self.best_fitness = self.fitness[self.best_idx]

    def gwo_update(self, agent, alpha, beta, delta, a):
        r1, r2 = np.random.rand(), np.random.rand()
        A1, C1 = 2 * a * r1 - a, 2 * r2
        D_alpha = abs(C1 * alpha - agent)
        X1 = alpha - A1 * D_alpha

        r1, r2 = np.random.rand(), np.random.rand()
        A2, C2 = 2 * a * r1 - a, 2 * r2
        D_beta = abs(C2 * beta - agent)
        X2 = beta - A2 * D_beta

        r1, r2 = np.random.rand(), np.random.rand()
        A3, C3 = 2 * a * r1 - a, 2 * r2
        D_delta = abs(C3 * delta - agent)
        X3 = delta - A3 * D_delta

        return (X1 + X2 + X3) / 3

    def scso_update(self, agent, best, t):
        SR = 2 - (2 * t / self.max_iter)
        rand_vec = np.random.uniform(-1, 1, agent.shape)
        direction = best - agent
        new_pos = agent + SR * rand_vec * direction

        if np.random.rand() < 0.1:
            new_pos = np.random.uniform(self.bounds[0], self.bounds[1], agent.shape)

        return np.clip(new_pos, self.bounds[0], self.bounds[1])

    def optimize(self):
        history = []
        for t in range(self.max_iter):
            a = 2 - 2 * t / self.max_iter
            sorted_idx = np.argsort(self.fitness)
            alpha = self.population[sorted_idx[0]]
            beta = self.population[sorted_idx[1]]
            delta = self.population[sorted_idx[2]]

            if self.mode == "GWO":
                new_population = np.array([
                    self.gwo_update(agent, alpha, beta, delta, a)
                    for agent in self.population
                ])
            else:
                new_population = np.array([
                    self.scso_update(agent, alpha, t)
                    for agent in self.population
                ])

            new_fitness = np.apply_along_axis(self.obj_func, 1, new_population)

            self.population = new_population
            self.fitness = new_fitness

            best_idx = np.argmin(self.fitness)
            if self.fitness[best_idx] < self.best_fitness:
                self.best_fitness = self.fitness[best_idx]
                self.best_solution = self.population[best_idx].copy()
                self.stagnation_counter = 0
            else:
                self.stagnation_counter += 1

            if self.mode == "GWO" and self.stagnation_counter >= self.stagnation_limit:
                self.mode = "SCSO"
                self.recovery_timer = self.recovery_duration
            elif self.mode == "SCSO":
                print("SCSO Triggered")
                self.recovery_timer -= 1
                if self.recovery_timer <= 0:
                    self.mode = "GWO"
                    self.stagnation_counter = 0

            history.append(self.best_fitness)

        return self.best_solution, self.best_fitness, history

# Run Adaptive SCSO-GWO Test
dimensions = [25, 50, 100, 250]
num_runs = 5
num_agents = 20
max_iterations = 100

print(f"ADAPTIVE GWO-SCSO - RASTRIGIN [{num_agents} Particles | {max_iterations} Iterations]")
print("--------------------------------------------------------------")
for dim in dimensions:
    best_fitnesses = []
    total_time = 0
    for run in range(num_runs):
        start = time.time()
        optimizer = AdaptiveGWOSCSO(rastrigin, dim, n_agents=num_agents, max_iter=max_iterations)
        _, best_fit, _ = optimizer.optimize()
        duration = time.time() - start
        print(f"#{run+1} Dimension: {dim} | Best Fitness: {best_fit:.5f} | Time: {duration:.2f}s")
        best_fitnesses.append(best_fit)
        total_time += duration

    avg_fit = sum(best_fitnesses) / num_runs
    min_fit = min(best_fitnesses)
    avg_time = total_time / num_runs
    print(f"Dimension: {dim} | Best: {min_fit:.5f} | Avg: {avg_fit:.5f} | Time: {avg_time:.2f}s")
    print("================================================================")


ADAPTIVE GWO-SCSO - RASTRIGIN [20 Particles | 100 Iterations]
--------------------------------------------------------------
#1 Dimension: 25 | Best Fitness: 0.00000 | Time: 0.29s
#2 Dimension: 25 | Best Fitness: 0.00000 | Time: 0.34s
#3 Dimension: 25 | Best Fitness: 0.00000 | Time: 0.45s
#4 Dimension: 25 | Best Fitness: 0.00000 | Time: 0.24s
#5 Dimension: 25 | Best Fitness: 0.00000 | Time: 0.39s
Dimension: 25 | Best: 0.00000 | Avg: 0.00000 | Time: 0.34s
#1 Dimension: 50 | Best Fitness: 0.00000 | Time: 0.25s
#2 Dimension: 50 | Best Fitness: 0.00000 | Time: 0.58s
#3 Dimension: 50 | Best Fitness: 0.00000 | Time: 0.55s
#4 Dimension: 50 | Best Fitness: 0.00000 | Time: 0.54s
#5 Dimension: 50 | Best Fitness: 0.00000 | Time: 0.47s
Dimension: 50 | Best: 0.00000 | Avg: 0.00000 | Time: 0.48s
#1 Dimension: 100 | Best Fitness: 0.00000 | Time: 0.25s
#2 Dimension: 100 | Best Fitness: 0.00000 | Time: 0.30s
#3 Dimension: 100 | Best Fitness: 0.00000 | Time: 0.43s
#4 Dimension: 100 | Best Fitness: 0.000